In [6]:
import pandas as pd
import yfinance as yf
import datetime as dt

In [7]:

"""
etf_categorize.py contains methods for categorizing ETFs based on Segment
"""


#1
def get_commodities(data):
    input_1 = "Commodities"

    series = data[data['Segment'].str.contains(input_1)]
    commodities = list(series['Ticker'])
    return commodities


#2
def get_global_non_US_equities(data):
    input_1 = "Equity"

    contains = ["Developed", "China", "Australia", "Russia", "South Korea", "Switzerland", "Taiwan", "U.K.", "Global"]
    not_contain = ": U.S. -"
    dev_markets = "Equity: Developed Markets Ex-U.S. - Total Market"
    # contains Equity and one of the string from "contains" array and does not contain ": U.S. -"
    series = data[data['Segment'].str.contains(input_1) & data['Segment'].str.contains('|'.join(contains)) & ~data['Segment'].str.contains(not_contain) & ~data['Segment'].str.contains(dev_markets)]
    global_equities = list(series['Ticker'])

    return global_equities


#3
def get_emerging_markets(data):
    input_1 = "Equity"
    input_2 = "Emerging Markets"
    series = data[data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)]
    emerging_market_equities = list(series['Ticker'])

    return emerging_market_equities


#4
def get_large_cap(data):
    input_1 = "Equity"
    input_2 = "Large Cap"

    series = data[data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)]
    stbonds = list(series['Ticker'])

    return stbonds


#5
def get_small_mid(data):
    input_1 = "Equity"
    input_2 = "Small Cap"
    input_3 = "Mid Cap"
    series = data[(data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)) | (data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_3))]
    real_estate_equities = list(series['Ticker'])

    return real_estate_equities

#6
def get_total_market_equities(data):
    input_1 = "Equity"
    input_2 = ": U.S. - Total Market"
    # , ": U.S. Aerospace & Defense", ": U.S. Banks", ": U.S. Basic Materials", ": U.S. Biotech", ": U.S. Consumer", ": U.S. Energy", ": U.S. Financial", ": U.S. Health", ": U.S. Homebuilding", ": U.S. Industrials", ": U.S. Internet", ": U.S. Technology", ": U.S. Telecommunications", ": U.S. Transportation", ": U.S. Utilities",
    series = data[data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)]
    stbonds = list(series['Ticker'])

    return stbonds

#7
def get_real_estate(data):
    input_1 = "Equity"
    input_2 = "Real Estate"
    input_3 = "REITs"
    series = data[(data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)) | (data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_3))]
    real_estate_equities = list(series['Ticker'])

    return real_estate_equities


#8
def get_total_bonds(data):
    input_1 = "Fixed Income"
    input_2 = "Global"
    input_3 = "U.S. - Broad Market"

    series = data[(data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)) | (data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_3))]
    stbonds = list(series['Ticker'])

    return stbonds


#9
def get_corporate_bonds(data):
    input_1 = "Fixed Income"
    input_2 = "Corporate"

    series = data[data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)]
    stbonds = list(series['Ticker'])

    return stbonds


#10
def get_government_bonds(data):
    input_1 = "Fixed Income"
    input_2 = "Government"

    series = data[data['Segment'].str.contains(input_1) & data['Segment'].str.contains(input_2)]
    stbonds = list(series['Ticker'])

    return stbonds

#
# 11
def get_int_dev_markets(data):
    input_1 = "Equity: Developed Markets Ex-U.S. - Total Market"

    series = data[data['Segment'].str.contains(input_1)]
    stbonds = list(series['Ticker'])

    return stbonds




In [8]:
def screen_etfs(dataset):

    # Dataset is now stored in a Pandas Dataframe
    accepted_grading = ['A', 'B']
    # return_volatility_grades = ['A+', 'A', 'A-', 'B+', 'B', 'B-']
    invalidIndex = dataset[(dataset['Expense Ratio'] == '--') | (dataset['AUM'] == '--')].index
    dataset.drop(invalidIndex, inplace=True)
    # eliminate dollar sign
    dataset['AUM'] = dataset['AUM'].str.replace('$', '')

    # convert to number with float
    for ind in dataset.index:
        if dataset['AUM'][ind][-1] == 'B':
            dataset['AUM'][ind] = dataset['AUM'][ind][:-1]
            dataset['AUM'][ind] = float(dataset['AUM'][ind][:-1]) * 1000000000

        elif dataset['AUM'][ind][-1] == 'M':
            dataset['AUM'][ind] = dataset['AUM'][ind][:-1]
            dataset['AUM'][ind] = float(dataset['AUM'][ind][:-1]) * 1000000

        elif dataset['AUM'][ind][-1] == 'K':
            dataset['AUM'][ind] = dataset['AUM'][ind][:-1]
            dataset['AUM'][ind] = float(dataset['AUM'][ind][:-1]) * 1000

    # remove inverse and leveraged
    for ind in dataset.index:
        if ('Inverse' in dataset['Segment'][ind]) or ('Leveraged' in dataset['Segment'][ind]):
            dataset = dataset.drop([ind])

    dataset['Expense Ratio'] = dataset['Expense Ratio'].str.replace('%', '')
    dataset['Expense Ratio'] = dataset['Expense Ratio'].astype(float)

    # rules
    AUM_test = dataset['AUM'].astype(float) >= 1000000000
    ER_test = dataset['Expense Ratio'] <= 0.75
    grade_test = dataset['Grade'].isin(accepted_grading)
    print(len(dataset))
    dataset = dataset[grade_test]
    dataset = dataset[AUM_test]
    dataset = dataset[ER_test]

    return dataset

In [9]:
dataset = pd.read_csv('NewETFdata.csv', header=0,
                          encoding='unicode_escape')

data = screen_etfs(dataset= dataset)

print(data)

new_data = get_government_bonds(data)

new_data

C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  if __name__ == '__main__':


1010
     Ticker                                               Name  \
660     FEX           First Trust Large Cap Core AlphaDEX Fund   
661    SDOG                      ALPS Sector Dividend Dogs ETF   
662     BWX  SPDR Bloomberg Barclays International Treasury...   
663    SPLB        SPDR Portfolio Long Term Corporate Bond ETF   
664    PNQI                        Invesco NASDAQ Internet ETF   
...     ...                                                ...   
1124    QQQ                                  Invesco QQQ Trust   
1125    VOO                               Vanguard S&P 500 ETF   
1126    VTI                    Vanguard Total Stock Market ETF   
1127    IVV                           iShares Core S&P 500 ETF   
1128    SPY                             SPDR S&P 500 ETF Trust   

                                                Segment       Issuer  \
660                            Equity: U.S. - Large Cap  First Trust   
661                            Equity: U.S. - Large Cap   

C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\ipykernel_launcher.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


['BWX',
 'JMST',
 'IGOV',
 'USFR',
 'CMF',
 'ITM',
 'FMB',
 'SPIP',
 'PZA',
 'BAB',
 'SPTI',
 'VWOB',
 'SPMB',
 'SPTL',
 'STIP',
 'SPTS',
 'TFI',
 'SCHR',
 'SUB',
 'SHM',
 'VGIT',
 'SCHO',
 'VGSH',
 'VTIP',
 'VTEB',
 'IEI',
 'BIL',
 'VMBS',
 'SCHP',
 'IEF',
 'SHV',
 'TLT',
 'EMB',
 'SHY',
 'MUB',
 'MBB',
 'TIP']

In [11]:
def get_data(ETF_list, number_of_years = 5):
    # Setting the date of today
    Current_Date = dt.datetime.today()

    # Setting the historical date (starting date)
    Historical_Date = dt.datetime.now() - dt.timedelta(days=number_of_years * 365)
    # You can set the date here, say like "2020-07-23"
    startdate = Historical_Date.strftime("%Y-%m-%d")
    enddate = Current_Date.strftime("%Y-%m-%d")

    print("Printing ETF list: {}".format(ETF_list))
    dflist = []
    for etf in ETF_list:
        try:
            df = yf.download(etf, start=startdate, end=enddate)
            df = df[['Adj Close']]
            df.rename(columns={'Adj Close': '{} Adj Close'.format(str(etf))}, inplace=True)
            dflist.append(df)
        except:
            print("Problem with yfinance download.")

    finaldf = pd.concat(dflist, axis=1)
    finaldf = finaldf.dropna(axis='columns')
    print("Final df is:")
    print(finaldf)
    return finaldf

In [12]:
values = get_data(new_data)

Printing ETF list: ['BWX', 'JMST', 'IGOV', 'USFR', 'CMF', 'ITM', 'FMB', 'SPIP', 'PZA', 'BAB', 'SPTI', 'VWOB', 'SPMB', 'SPTL', 'STIP', 'SPTS', 'TFI', 'SCHR', 'SUB', 'SHM', 'VGIT', 'SCHO', 'VGSH', 'VTIP', 'VTEB', 'IEI', 'BIL', 'VMBS', 'SCHP', 'IEF', 'SHV', 'TLT', 'EMB', 'SHY', 'MUB', 'MBB', 'TIP']
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



[*********************100%***********************]  1 of 1 completed

C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,



[*********************100%***********************]  1 of 1 completed


...

C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


[*********************100%***********************]  1 of 1 completed


C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


[*********************100%***********************]  1 of 1 completed
Final df is:
            BWX Adj Close  IGOV Adj Close  USFR Adj Close  CMF Adj Close  \
Date                                                                       
2016-06-06      27.082169       48.747730       23.560335      54.389580   
2016-06-07      27.182878       48.910423       23.560335      54.425735   
2016-06-08      27.288389       49.063274       23.560335      54.529694   
2016-06-09      27.202055       48.954796       23.692959      54.602020   
2016-06-10      27.130123       48.747730       23.692959      54.728569   
...                   ...             ...             ...            ...   
2021-05-27      29.906969       53.419998       25.100000      62.449024   
2021-05-28      29.866999       53.380001       25.100000      62.468998   
2021-06-01      29.940001       53.540001       25.110001      62.480000   
2021-06-02      29.959999       53.430000       25.110001      62.490002   
2021-0

C:\Users\Admin\anaconda3\envs\nlp_course\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
values.head()

,BWX Adj Close,IGOV Adj Close,USFR Adj Close,CMF Adj Close,ITM Adj Close,FMB Adj Close,SPIP Adj Close,PZA Adj Close,BAB Adj Close,SPTI Adj Close,...,VMBS Adj Close,SCHP Adj Close,IEF Adj Close,SHV Adj Close,TLT Adj Close,EMB Adj Close,SHY Adj Close,MUB Adj Close,MBB Adj Close,TIP Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2016-06-06,27.082169,48.747730,23.560335,54.389580,43.650139,47.117020,25.070551,22.615705,25.691917,28.400925,...,47.860394,50.497444,101.489342,104.400894,118.434822,88.997314,79.749321,100.320580,96.856636,103.912743
2016-06-07,27.182878,48.910423,23.560335,54.425735,43.739643,47.258194,25.158037,22.641693,25.733503,28.447485,...,47.842533,50.669586,101.636162,104.410385,118.721413,89.140068,79.768066,100.383049,96.891991,104.192375
2016-06-08,27.288389,49.063274,23.560335,54.529694,43.739643,47.238789,25.201784,22.667673,25.716869,28.456804,...,47.905029,50.778290,101.755432,104.410385,119.357323,89.608002,79.786850,100.490143,96.856636,104.435959
2016-06-09,27.202055,48.954796,23.692959,54.602020,43.793331,47.311134,25.232403,22.710976,25.791727,28.475428,...,47.789001,50.805481,101.911407,104.410385,120.127502,89.496956,79.805626,100.659729,96.971657,104.472031
2016-06-10,27.130123,48.747730,23.692959,54.728569,43.900707,47.337608,25.236778,22.762951,25.924803,28.512669,...,47.869312,50.814537,102.269302,104.419823,120.709686,89.155922,79.852600,100.829338,96.989334,104.472031


In [14]:
stocks = values


In [15]:
stocks.pct_change(1).mean()

BWX Adj Close     0.000086
IGOV Adj Close    0.000078
USFR Adj Close    0.000051
CMF Adj Close     0.000118
ITM Adj Close     0.000151
FMB Adj Close     0.000160
SPIP Adj Close    0.000171
PZA Adj Close     0.000160
BAB Adj Close     0.000212
SPTI Adj Close    0.000104
VWOB Adj Close    0.000206
SPMB Adj Close    0.000100
SPTL Adj Close    0.000157
STIP Adj Close    0.000116
SPTS Adj Close    0.000059
TFI Adj Close     0.000128
SCHR Adj Close    0.000095
SUB Adj Close     0.000062
SHM Adj Close     0.000067
VGIT Adj Close    0.000096
SCHO Adj Close    0.000066
VGSH Adj Close    0.000066
VTIP Adj Close    0.000114
VTEB Adj Close    0.000136
IEI Adj Close     0.000092
BIL Adj Close     0.000038
VMBS Adj Close    0.000089
SCHP Adj Close    0.000168
IEF Adj Close     0.000099
SHV Adj Close     0.000045
TLT Adj Close     0.000160
EMB Adj Close     0.000199
SHY Adj Close     0.000063
MUB Adj Close     0.000128
MBB Adj Close     0.000090
TIP Adj Close     0.000164
dtype: float64

In [17]:
import numpy as np

In [18]:
log_return = np.log(stocks/stocks.shift(1))

In [20]:
log_return.head()

,BWX Adj Close,IGOV Adj Close,USFR Adj Close,CMF Adj Close,ITM Adj Close,FMB Adj Close,SPIP Adj Close,PZA Adj Close,BAB Adj Close,SPTI Adj Close,...,VMBS Adj Close,SCHP Adj Close,IEF Adj Close,SHV Adj Close,TLT Adj Close,EMB Adj Close,SHY Adj Close,MUB Adj Close,MBB Adj Close,TIP Adj Close
Date,,,,,,,,,,,,,,,,,,,,,
2016-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-06-07,0.003712,0.003332,0.000000,0.000665,0.002048,0.002992,0.003484,0.001148,0.001617,0.001638,...,-0.000373,0.003403,0.001446,0.000091,0.002417,0.001603,0.000235,0.000623,0.000365,0.002687
2016-06-08,0.003874,0.003120,0.000000,0.001908,0.000000,-0.000411,0.001737,0.001147,-0.000647,0.000328,...,0.001305,0.002143,0.001173,0.000000,0.005342,0.005236,0.000235,0.001066,-0.000365,0.002335
2016-06-09,-0.003169,-0.002213,0.005613,0.001325,0.001227,0.001530,0.001214,0.001908,0.002907,0.000654,...,-0.002425,0.000535,0.001532,0.000000,0.006432,-0.001240,0.000235,0.001686,0.001187,0.000345
2016-06-10,-0.002648,-0.004239,0.000000,0.002315,0.002449,0.000559,0.000173,0.002286,0.005146,0.001307,...,0.001679,0.000178,0.003506,0.000090,0.004835,-0.003818,0.000588,0.001684,0.000182,0.000000


In [21]:
len(log_return.columns)

36

In [22]:
test_tup = (0,1)
bounds = ((test_tup,) * len(log_return.columns))

In [25]:
# The minimizing algorithm

In [26]:
from scipy.optimize import minimize

In [27]:
def get_ret_vol_sr(weights): 
    weights = np.array(weights)
    ret = np.sum(log_return.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T,np.dot(log_return.cov()*252,weights)))
    sr = ret/vol 
    return np.array([ret,vol,sr])

In [28]:
def neg_sharpe(weights): 
    return get_ret_vol_sr(weights)[2] * -1

# check allocation sums to 1
def check_sum(weights): 
    return np.sum(weights) - 1

# create constraint variable
cons = ({'type':'eq','fun':check_sum})

In [33]:
init_guess = [1/len(log_return.columns)] * len(log_return.columns)

init_guess

[0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776]

In [34]:
opt_results = minimize(neg_sharpe, init_guess,bounds = bounds, method='SLSQP', constraints=cons)

opt_results.x

array([0.00000000e+00, 0.00000000e+00, 3.73255049e-02, 0.00000000e+00,
       3.30717060e-11, 3.02922595e-03, 0.00000000e+00, 1.05579261e-11,
       2.90213086e-11, 2.36315840e-13, 5.11019776e-04, 0.00000000e+00,
       0.00000000e+00, 1.27779002e-02, 2.01341993e-11, 2.78631807e-12,
       0.00000000e+00, 2.49672570e-11, 1.45375978e-11, 0.00000000e+00,
       3.14912927e-11, 2.89865794e-11, 1.44503169e-03, 2.46274917e-11,
       0.00000000e+00, 4.64076744e-01, 2.23218693e-11, 0.00000000e+00,
       0.00000000e+00, 4.80834574e-01, 0.00000000e+00, 1.54147920e-11,
       2.84009593e-11, 2.02266731e-11, 3.73730368e-12, 0.00000000e+00])

In [32]:
get_ret_vol_sr(opt_results.x)

array([1.09645888e-02, 1.82070576e-03, 6.02216408e+00])